In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim

import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [ ]:
class GCNLayer(nn.Module):
    def __init__(self, c_in, c_out):
        super().__init__()
        self.projection = nn.Linear(c_in, c_out)

    def forward(self, node_feat, adj_mat):
        num_neighbors = adj_mat.sum(dim = -1, keepdims=True)
        node_feat = self.projection(node_feat)
        node_feat = torch.bmm(adj_mat, node_feat)
        node_feat = node_feat / num_neighbors
        return node_feat

In [ ]:
import networkx as nx

nx_g = nx.from_edgelist([(2,1), (2,3), (4,2), (3,4)])

import matplotlib.pyplot as plt
fig = plt.figure()
nx.draw(nx_g, with_labels=True)

In [ ]:
node_feats = torch.arange(8, dtype=torch.float32).view(1, 4, 2)

adj_matrix = torch.Tensor([[[1, 1, 0, 0],
                            [1, 1, 1, 1],
                            [0, 1, 1, 1],
                            [0, 1, 1, 1]]])

print(node_feats)

print(adj_matrix)

In [ ]:
layer = GCNLayer(c_in=2, c_out=2)

layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])
layer.projection.bias.data = torch.Tensor([0., 0.])

with torch.no_grad():
    out_feats = layer(node_feats, adj_matrix)

print(out_feats)

# GAT

In [ ]:
class GATLayer(nn.Module):
    def __init__(self, c_in, c_out, num_heads = 1, concat_heads = True, alpha = 0.2) -> None:
        super().__init__()
        self.num_heads = num_heads
        self.concat_heads = concat_heads

        if self.concat_heads:
            assert c_out % num_heads == 0
            c_out = c_out // num_heads
        
        self.projection = nn.Linear(c_in, c_out*num_heads)
        self.a = nn.Parameter(torch.Tensor(num_heads, 2 * c_out))
        self.leakyrelu = nn.LeakyReLU(alpha)

        nn.init.xavier_uniform_(self.projection.weight.data, gain = 1.414)
        nn.init.xavier_uniform_(self.a.data, gain = 1.414)

    def forward(self, node_feats, adj_matrix, print_attn_probs=False):
        batch_size, num_nodes = node_feats.size(0), node_feats.size(1)

        node_feats = self.projection(node_feats)
        node_feats = node_feats.view(batch_size, num_nodes, self.num_heads, -1)

        edges = adj_matrix.nonzero(as_tuple = False)

        node_feats_flat = node_feats.view(batch_size*num_nodes, self.num_heads, -1)

        edge_indices_row = edges[:, 0] * num_nodes + edges[:, 1]
        edge_indices_col = edges[:, 0] * num_nodes + edges[:, 2]
        a_input = torch.cat([
            torch.index_select(input=node_feats_flat, index=edge_indices_row, dim = 0),
            torch.index_select(input=node_feats_flat, index=edge_indices_col, dim = 0)
        ], dim = -1)

        attn_logits = torch.einsum('bhc,hc->bh', a_input, self.a)
        attn_logits = self.leakyrelu(attn_logits)

        attn_matrix = attn_logits.new_zeros(adj_matrix.shape+(self.num_heads,)).fill_(-9e15)
        attn_matrix[adj_matrix[...,None].repeat(1,1,1,self.num_heads) == 1] = attn_logits.reshape(-1)

        attn_probs = F.softmax(attn_matrix, dim = 2)
        if print_attn_probs:
            print(attn_probs.permute(0, 3, 1, 2))
        node_feats = torch.einsum('bijh,bjhc->bihc', attn_probs, node_feats)

        if self.concat_heads:
            node_feats = node_feats.reshape(batch_size, num_nodes, -1)
        else:
            node_feats = node_feats.mean(dim=2)

        return node_feats


In [ ]:
layer = GATLayer(c_in=2, c_out=2, num_heads=2)

layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])
layer.projection.bias.data = torch.Tensor([0., 0.])

layer.a.data = torch.Tensor([[-0.2, 0.3], [0.1, -0.1]])

with torch.no_grad():
    out_feats = layer(node_feats, adj_matrix, print_attn_probs = True)

print(out_feats)

# Pytorch Geometric

In [ ]:
import torch_geometric
import torch_geometric.nn as g_nn
import torch_geometric.data as g_data

In [ ]:
import torch_geometric.datasets
from torch_geometric.transforms import NormalizeFeatures

dataset = torch_geometric.datasets.Planetoid(root='./data', name='Cora', transform=NormalizeFeatures())

In [ ]:
import numpy as np

data = dataset[0]
print(data['edge_index'])
print(data)

In [ ]:
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, hidden_chan):
        super().__init__()
        torch.manual_seed(1123)
        self.conv1 = GCNConv(dataset.num_features, hidden_chan)
        self.conv2 = GCNConv(hidden_chan, dataset.num_classes)
    
    def forward(self, x, edge_index):
        
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x
    
model = GCN(hidden_chan=16)
print(model)

In [ ]:
model = GCN(hidden_chan=16)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train(model):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

def test(model):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    test_correct = pred[data.test_mask] == data.y[data.test_mask]
    test_acc = int(test_correct.sum()) / int(data.test_mask.sum())
    return test_acc

for epoch in range(1, 101):
    loss = train(model)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

In [ ]:
test_acc = test()
print(test_acc)

In [ ]:
from torch_geometric.nn import GATConv

class GAT(torch.nn.Module):
    def __init__(self, hidden_chan, heads):
        super().__init__()
        torch.manual_seed(1123)
        self.conv1 = GATConv(dataset.num_features, hidden_chan)
        self.conv2 = GATConv(hidden_chan, dataset.num_classes)
    
    def forward(self, x, edge_index):
        print(x, edge_index)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        raise Exception
        return x

def train(model):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

def test(model, mask):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    test_correct = pred[mask] == data.y[mask]
    test_acc = int(test_correct.sum()) / int(mask.sum())
    return test_acc
    
gatmodel = GAT(hidden_chan=8, heads=8)
optimizer = torch.optim.Adam(gatmodel.parameters(), lr = 0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
print(gatmodel)

In [ ]:
for epoch in range(1, 200):
    loss = train(gatmodel)
    val_acc = test(gatmodel, data.val_mask)
    test_acc = test(gatmodel, data.test_mask)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val; {val_acc:.4f}, \
          Test: {test_acc:.4f}')

In [ ]:
test_acc = test(gatmodel, mask=data.train_mask)
print(test_acc)

In [ ]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())


In [ ]:
print(data['edge_index'])

In [ ]:
from torch_geometric.utils import degree

row, col = data['edge_index']
print(row, col)
deg = degree

### nodes
* ports
	* id
* vessels
	* id
	* fish		
	* maybe? location
	* label (one hot vector)
	* maybe? country
	* ask for information about vessel types		

### edges
* port
* ectr
	

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import time

# nx_g = nx.from_edgelist([(2,1), (2,3), (4,2), (3,4)])

# import matplotlib.pyplot as plt
# fig = plt.figure()
# nx.draw(nx_g, with_labels=True)

data_events = pd.read_csv('data/Starboard/events.csv')
data_vessels = pd.read_csv('data/Starboard/vessels.csv')

t0 = time.time()
# Create Nodes and Edges
G = nx.Graph()

num_vessels = 0
n_colormap = {}
feature_dict = {x['vessel_id']: x['label'] for _, x in data_vessels.iterrows()}
node_att_map = {node: ind for ind, node in enumerate(set(data_vessels['label']))}
node_att_map['Port'] = len(node_att_map)
node_att_map['NA'] = len(node_att_map)
node_att_map['Fish'] = len(node_att_map)
edge_att_map = {'ECTR': 0, 'FISH': 1, 'PORT': 2}
eye = np.eye(len(node_att_map))

for ind, data in data_events.iterrows():
    init_vessel = data['vessel_id']
    sec_vessel = data['vessel_id2']
    port = data['port_id']
    event = data['event_type']

    G.add_node(init_vessel, name = 'vessel', label = feature_dict[init_vessel])
    n_colormap[init_vessel] = 'blue'
    
    # if not np.isnan(sec_vessel):
    if not np.isnan(sec_vessel):# and sec_vessel in feature_dict:
        if sec_vessel in feature_dict:
            label = feature_dict[sec_vessel]
            G.add_node(sec_vessel, name = 'vessel', label = label,
                       one_hot = eye[node_att_map[label]])
        else:
            G.add_node(sec_vessel, name = 'vessel', label = 'NA',
                       one_hot = eye[node_att_map['NA']])

        G.add_edge(init_vessel, sec_vessel, event = event, color = 'blue')
        n_colormap[sec_vessel] = 'blue'

    elif not np.isnan(port):
        G.add_node(port, name = 'port', label = 'Port',
                   one_hot = eye[node_att_map['Port']])
        G.add_edge(init_vessel, port, event = 'PORT', color = 'orange')
        n_colormap[port] = 'orange'
    
    else:
        G.add_edge(init_vessel, 0, event = event, color = 'green')
        n_colormap[0] = 'green'

G.nodes[0]['one_hot'] = eye[node_att_map['Fish']]

mapping = {node: ind for ind, node in enumerate(G.nodes)}
rev_map = {ind: node for ind, node in enumerate(G.nodes)}

H = nx.relabel_nodes(G, mapping, copy = True)

node_att = nx.get_node_attributes(H, 'one_hot')
edge_att = nx.get_edge_attributes(H, 'event')

x = np.array(list(node_att.values()))
edge_index = np.array(H.edges()).T

import matplotlib.pyplot as plt
pos = nx.spring_layout(G, seed = 1123)
plt.figure(1, figsize=(10,10))
e_colors = [G[u][v]['color'] for u,v in G.edges()]
nx.draw(G, pos, node_color = n_colormap.values(), edge_color = e_colors, node_size=50, alpha=0.5, with_labels = False, font_size = 8, font_weight = 'bold')
plt.axis('off')
plt.show()


t1 = time.time()

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import time

# nx_g = nx.from_edgelist([(2,1), (2,3), (4,2), (3,4)])

# import matplotlib.pyplot as plt
# fig = plt.figure()
# nx.draw(nx_g, with_labels=True)

data_vessels = pd.read_csv('data/Starboard/vessels.csv')

t0 = time.time()
# Create Nodes and Edges
G = nx.Graph()

num_vessels = 0
n_colormap = {}
feature_dict = {x['vessel_id']: x['label'] for _, x in data_vessels.iterrows()}
node_att_map = {node: ind for ind, node in enumerate(set(data_vessels['label']))}
node_att_map['Port'] = len(node_att_map)
node_att_map['NA'] = len(node_att_map)
node_att_map['Fish'] = len(node_att_map)
edge_att_map = {'ECTR': 0, 'FISH': 1, 'PORT': 2}
eye = np.eye(len(node_att_map))

def add_to_g(path, G, right = True):
    data_events = pd.read_csv(path)
    for _, data in data_events.iterrows():
        init_vessel = data['vessel_id1']
        sec_vessel = data['vessel_id2']
        event = data['event_type']

        G.add_node(init_vessel, name = 'vessel', label = feature_dict[init_vessel])
        n_colormap[init_vessel] = 'blue'
        
        # if not np.isnan(sec_vessel):
        if event == 'ECTR':
            if sec_vessel in feature_dict:
                label = feature_dict[sec_vessel]
                G.add_node(sec_vessel, name = 'vessel', label = label,
                        one_hot = eye[node_att_map[label]])
            else:
                G.add_node(sec_vessel, name = 'vessel', label = 'NA',
                        one_hot = eye[node_att_map['NA']])

            if right:
                G.add_edge(init_vessel, sec_vessel, event = event, color = 'blue')
            else:
                G.add_edge(init_vessel, sec_vessel, event = event, color = 'lightblue')
            n_colormap[sec_vessel] = 'blue'

        elif event == 'PORT':
            G.add_node(sec_vessel, name = 'port', label = 'Port',
                    one_hot = eye[node_att_map['Port']])
            
            if right:
                G.add_edge(init_vessel, sec_vessel, event = 'PORT', color = 'red')
            else:
                G.add_edge(init_vessel, sec_vessel, event = 'PORT', color = 'pink')
            n_colormap[sec_vessel] = 'red'
        
        else:
            if right:
                G.add_edge(init_vessel, sec_vessel, event = event, color = 'green')
            else:
                G.add_edge(init_vessel, sec_vessel, event = event, color = 'lightgreen')
            n_colormap[0] = 'green'

path1 = 'res/right_data.csv'
add_to_g(path1, G, True)
path2 = 'res/wrong_data.csv'
add_to_g(path2, G, False)

G.nodes[0]['one_hot'] = eye[node_att_map['Fish']]

mapping = {node: ind for ind, node in enumerate(G.nodes)}
rev_map = {ind: node for ind, node in enumerate(G.nodes)}

H = nx.relabel_nodes(G, mapping, copy = True)

node_att = nx.get_node_attributes(H, 'one_hot')
edge_att = nx.get_edge_attributes(H, 'event')

x = np.array(list(node_att.values()))
edge_index = np.array(H.edges()).T

import matplotlib.pyplot as plt
pos = nx.spring_layout(G, seed = 1123)
plt.figure(1, figsize=(10,10))
e_colors = [G[u][v]['color'] for u,v in G.edges()]
nx.draw(G, pos, node_color = n_colormap.values(), edge_color = e_colors, node_size=50, alpha=0.7, with_labels = False, font_size = 8, font_weight = 'bold')
plt.axis('off')
plt.show()


t1 = time.time()

In [ ]:
import os.path as osp

import torch
from sklearn.metrics import average_precision_score, roc_auc_score, root_mean_squared_error
from torch.nn import Linear

from torch_geometric.data.temporal import TemporalData
from torch_geometric.datasets import JODIEDataset
from torch_geometric.loader import TemporalDataLoader
from torch_geometric.nn import TGNMemory, TransformerConv
from torch_geometric.nn.models.tgn import (
    IdentityMessage,
    LastAggregator,
    LastNeighborLoader,
)
from torch.utils.tensorboard import SummaryWriter

import datetime
import pandas as pd
import numpy as np
import pyarrow
import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Starboard data
events = ['FISH', 'PORT', 'ECTR']
event_dict = dict(zip(events, range(len(events))))
rev_event_dict = dict(zip(range(len(events)), events))
data_events = pd.read_parquet('data/Starboard/events.parquet')
data_vessels = pd.read_csv('data/Starboard/vessels.csv')
feature_dict = {x['vessel_id']: x['label'] for _, x in data_vessels.iterrows()}

# Initialise inputs 
src = np.zeros((len(data_events)))
dst = np.zeros((len(data_events)))
t = np.zeros((len(data_events)))
# features = (lat_n,lat_s,lon_e,lon_w,ves1,ves2)
features = np.zeros((len(data_events), 4))
y = np.zeros((len(data_events), len(events)))

# convert timesteps
data_events['start_time'] = pd.to_datetime(data_events['start_time'], format='%d/%m/%Y %H:%M')
data_events = data_events.sort_values(by='start_time').reset_index(drop=True)
timesteps = data_events['start_time'].dt.dayofyear

# add fish to events
data_events['event_type'] = data_events['event_type'].fillna('PORT')

prog_bar = tqdm.tqdm(range(len(data_events)))
for ind, data in data_events.iterrows():
    # Define source and dest array
    src[ind] = data['vessel_id']

    if not np.isnan(data['vessel_id2']):
        dst[ind] = data['vessel_id2']
    elif not np.isnan(data['port_id']):
        dst[ind] = data['port_id']
    else:
        dst[ind] = 0

    # Timestamp
    t[ind] = timesteps[ind]

    # Features
    features[ind] = np.array([data['lat_n'], data['lat_s'], data['lon_e'], data['lon_w']])

    # Even types
    event = data['event_type']
    y[ind][event_dict[event]] = 1
    prog_bar.update(1)
    
prog_bar.close()

vals, indexes = np.unique(np.concatenate((src, dst)), return_inverse=True)
# print(src)
# print(dst)
src, dst = np.split(indexes, 2)
features = np.nan_to_num(features)

src = torch.Tensor(src).type(torch.long)
dst = torch.Tensor(dst).type(torch.long)
t = torch.Tensor(t).type(torch.long)
features = torch.Tensor(features)
y = torch.Tensor(y)

data = TemporalData(src, dst, t, features, y=y)

print(data)
# print(features)
# _, uniq = np.unique(pd.concat((data_events['vessel_id'], data_events['vessel_id2'], data_events['port_id'])), return_inverse=True)
# print(indexes)
# print(uniq)
# print(np.array_equal(uniq, indexes))
# vals, counts = np.unique(dst, return_counts=True)
# print(counts)
# print(vals)

In [ ]:
print(np.max(data_events['vessel_id']), np.max(data_events['vessel_id2']))

val_ratio = 0.1
test_ratio = 0.1
batch_size = 5000

train_data, val_data, test_data = data.train_val_test_split(
    val_ratio=val_ratio, test_ratio=test_ratio)

train_loader = TemporalDataLoader(
    train_data,
    batch_size=batch_size,
    neg_sampling_ratio=1.0,
)
val_loader = TemporalDataLoader(
    val_data,
    batch_size=batch_size,
    neg_sampling_ratio=1.0,
)
test_loader = TemporalDataLoader(
    test_data,
    batch_size=batch_size,
    neg_sampling_ratio=1.0,
)

torch.sum(test_data.y, dim=0) / torch.sum(test_data.y)

In [ ]:
import os.path as osp

import torch
from sklearn.metrics import average_precision_score, roc_auc_score
from torch.nn import Linear

from torch_geometric.data.temporal import TemporalData
from torch_geometric.datasets import JODIEDataset
from torch_geometric.loader import TemporalDataLoader
from torch_geometric.nn import TGNMemory, TransformerConv
from torch_geometric.nn.models.tgn import (
    IdentityMessage,
    LastAggregator,
    LastNeighborLoader,
)

import networkx as nx
import pandas as pd
import numpy as np
import copy
import scipy.sparse as sparse
import tqdm
import pyarrow

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Starboard data
events = ['FISH', 'PORT', 'ECTR']
event_dict = dict(zip(events, range(len(events))))
data_events = pd.read_parquet('data/Starboard/events.parquet')
data_vessels = pd.read_csv('data/Starboard/vessels.csv')
feature_dict = {x['vessel_id']: x['label'] for _, x in data_vessels.iterrows()}

# convert timesteps
data_events['start_time'] = pd.to_datetime(data_events['start_time'], format='%d/%m/%Y %H:%M')
data_events = data_events.sort_values(by='start_time').reset_index(drop=True)
timesteps = (data_events['start_time'] - data_events['start_time'][0]).dt.days

# add fish to events
data_events['event_type'] = data_events['event_type'].fillna('PORT')
data_events['ectr_id'] = data_events['ectr_id'].astype('Int64')
data_events['vessel_id2'] = data_events['vessel_id2'].astype('Int64')
data_events

In [ ]:
timesteps

In [ ]:
import pickle

with open('data/FB/adj_time_list.pickle', 'rb') as handle:
    adj_time_list = pickle.load(handle, encoding='latin1')

with open('data/FB/adj_orig_dense_list.pickle', 'rb') as handle:
    adj_orig_dense_list = pickle.load(handle, encoding='bytes')

print(adj_time_list)

print(adj_orig_dense_list)

In [ ]:
from torch.autograd import Variable

x_in_list = []
for i in range(0, 3):
    x_temp = torch.tensor(np.eye(663).astype(np.float32))
    x_in_list.append(torch.tensor(x_temp))

x_in = Variable(torch.stack(x_in_list))
x_in[0]

h_dim = 32
x_dim = 663

phi_x = torch.nn.Sequential(torch.nn.Linear(x_dim, h_dim), torch.nn.ReLU())

In [ ]:
hu = phi_x(x_in[0])
print(hu.shape)

In [ ]:
pd.unique(data_events['event_type'])
print(event_dict)
data_events.head(5)

# Create the data for VGRNN

In [3]:
adj_time_list = []
orig_list = []
all_feats = []
cur_date = timesteps[0]
ind_map = {}
rev_ind_map = {}

rows = []
cols = []
features = []
feature_lengths = []
print(cur_date)

length = 11510

prog_bar = tqdm.tqdm(range(len(data_events)))
for ind, data in data_events.iterrows():
    # Get event
    event = data['event_type']

    # Append new vessel
    if data['vessel_id'] not in ind_map:
        rev_ind_map[len(ind_map)] = data['vessel_id']
        ind_map[data['vessel_id']] = len(ind_map)
        features.append([0, 0, 0])

    # Create new vessels / port depending on encounter type
    match event:
        case 'ECTR':
            if data['vessel_id2'] not in ind_map:
                rev_ind_map[len(ind_map)] = data['vessel_id2']
                ind_map[data['vessel_id2']] = len(ind_map)
                features.append([0, 0, 0])
        case 'PORT':
            if data['port_id'] not in ind_map:
                rev_ind_map[len(ind_map)] = data['port_id']
                ind_map[data['port_id']] = len(ind_map)
                features.append([0, 0, 0])
        case 'FISH':
            pass
    
    # Create the adj list
    if cur_date == timesteps[ind]:
        rows.append(ind_map[data['vessel_id']])
        features[ind_map[data['vessel_id']]][event_dict[event]] = 1

        match event:
            case 'ECTR':
                cols.append(ind_map[data['vessel_id2']])
                features[ind_map[data['vessel_id2']]][event_dict[event]] = 1
            case 'PORT':
                cols.append(ind_map[data['port_id']])
                features[ind_map[data['port_id']]][event_dict[event]] = 1
            case 'FISH':
                cols.append(ind_map[data['vessel_id']])
                features[ind_map[data['vessel_id']]][event_dict[event]] = 1
        
        # Change this to end of timestep
        if ind + 1 == len(timesteps):
            all_feats.append(copy.deepcopy(features))

            np_row = torch.tensor(rows)
            np_cols = torch.tensor(cols)
            indices = torch.unique(torch.vstack((np_row, np_cols)), dim=1)
            ones = torch.ones(indices.shape[1])

            mat = torch.sparse_coo_tensor(indices, ones, (length, length))

            feature_lengths.append(len(features))
            # mat_dense = torch.Tensor(mat.todense())
            # orig_list.append(mat_dense)
            adj_time_list.append(mat)

    else:
        all_feats.append(copy.deepcopy(features))
        features = [[0, 0, 0] for _ in range(len(features))]

        np_row = torch.tensor(rows)
        np_cols = torch.tensor(cols)
        indices = torch.unique(torch.vstack((np_row, np_cols)), dim=1)
        ones = torch.ones(indices.shape[1])

        mat = torch.sparse_coo_tensor(indices, ones, (length, length))

        feature_lengths.append(len(features))
        # mat_dense = torch.Tensor(mat.todense())
        # orig_list.append(mat_dense)
        adj_time_list.append(mat)

        rows = []
        cols = []
        cur_date = timesteps[ind]

        rows.append(ind_map[data['vessel_id']])
        features[ind_map[data['vessel_id']]][event_dict[event]] = 1

        match event:
            case 'ECTR':
                cols.append(ind_map[data['vessel_id2']])
                features[ind_map[data['vessel_id2']]][event_dict[event]] = 1
            case 'PORT':
                cols.append(ind_map[data['port_id']])
                features[ind_map[data['port_id']]][event_dict[event]] = 1
            case 'FISH':
                cols.append(ind_map[data['vessel_id']])
                features[ind_map[data['vessel_id']]][event_dict[event]] = 1
    prog_bar.update(1)
prog_bar.close()

print(cur_date)
print(adj_time_list)
print(feature_lengths)
print(len(all_feats))

0


100%|██████████| 4687556/4687556 [05:08<00:00, 15208.29it/s]


729
[tensor(indices=tensor([[   0,    1,    2,  ..., 2769, 2770, 2773],
                       [   0,    1,    2,  ..., 2769, 2771, 2774]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(11510, 11510), nnz=2877, layout=torch.sparse_coo), tensor(indices=tensor([[   0,    1,    2,  ..., 3437, 3439, 3440],
                       [   0,    1,    2,  ..., 3438, 3439, 2921]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(11510, 11510), nnz=2300, layout=torch.sparse_coo), tensor(indices=tensor([[   0,    2,    3,  ..., 3882, 3883, 3884],
                       [   0,    2,    3,  ..., 3882,  866, 3884]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(11510, 11510), nnz=2347, layout=torch.sparse_coo), tensor(indices=tensor([[   0,    1,    2,  ..., 4266, 4268, 4269],
                       [   0,    1,    2,  ..., 1180, 2110, 4269]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(11510, 11510), nnz=2351, layou

In [35]:
abs(torch.eye(adj_time_list[0].shape[0])*adj_time_list[0] - adj_time_list[0])

tensor(indices=tensor([[   0,    1,    2,  ..., 2769, 2770, 2773],
                       [   0,    1,    2,  ..., 2769, 2771, 2774]]),
       values=tensor([0., 0., 0.,  ..., 0., 1., 1.]),
       size=(11510, 11510), nnz=2877, layout=torch.sparse_coo)

In [ ]:
rows = [1,2,2,3,4,5]
cols = [5,4,4,3,2,1]

np_row = torch.tensor(rows)
np_cols = torch.tensor(cols)
indices = torch.unique(torch.vstack((np_row, np_cols)), dim=1)
ones = torch.ones(indices.shape[1])

mat = torch.sparse_coo_tensor(indices, ones, (length, length))
mat = mat.to_sparse_csc()
mat


In [ ]:
mat - mat

In [12]:
import pickle

with open('data/Starboard/adj_time_list.pickle', 'wb') as f:
    pickle.dump(adj_time_list, f)
f.close()

In [ ]:
import scipy.sparse as sp
adjs_list = adj_time_list

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

def ismember(a, b, tol=5):
    rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
    return np.any(rows_close)

pbar = tqdm.tqdm(total=len(adjs_list))
false_edges_l, pos_edges_l = [], []
for i in range(1, len(adjs_list)):
    print(len(adjs_list[i].data))

    condition = ((adjs_list[i] - adjs_list[i-1])>0).todense().nonzero()
    num_false = int(len(condition[0]))
    
    adj = adjs_list[i]
    # # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()
    # Check that diag is zero:
    assert np.diag(adj.todense()).sum() == 0
    
    adj_triu = sp.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]
    edges_all = sparse_to_tuple(adj)[0]
    
    edges_false = []
    while len(edges_false) < num_false:
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], edges_all):
            continue
        if edges_false:
            if ismember([idx_j, idx_i], np.array(edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(edges_false)):
                continue
        edges_false.append([idx_i, idx_j])
    
    assert ~ismember(edges_false, edges_all)
    
    false_edges_l.append(np.asarray(edges_false))
    pos_edges_l.append(condition)

    pbar.update(1)
pbar.close()

In [13]:
with open('data/Starboard/orig_list.pickle', 'wb') as f:
    pickle.dump(orig_list, f)
f.close()

In [ ]:
import pickle

with open('data/Starboard/adj_time_list.pickle', 'rb') as handle:
    adj_time_list = pickle.load(handle)

with open('data/Starboard/orig_list.pickle', 'rb') as handle:
    adj_orig_dense_list = pickle.load(handle)

In [19]:
import scipy
import scipy.sparse

matr = adj_time_list[0]
G = nx.from_scipy_sparse_array(matr)


In [ ]:
half = len(feature_lengths)/2
print(half)
feature_lengths_arr = np.array(feature_lengths[:int(half)])
print(np.unique(np.diff(feature_lengths_arr), return_index=True))

In [23]:
orig_list[0][orig_list[0] != 0] = 1
orig_list[0]

for orig in orig_list:
    orig[orig != 0] = 1

In [ ]:
orig_list

In [ ]:
import numpy as np

a = np.array([1, 2, 3, 4])
b = np.array([5, 6, 7, 8])
c = np.array([9,10,11,12])

print(np.vstack((a, b, c)).T)

In [ ]:
final_dataset = final_dataset.to(device)

train_data, val_data, test_data = final_dataset.train_val_test_split(
    val_ratio=0.15, test_ratio=0.15)

train_loader = TemporalDataLoader(
    train_data,
    batch_size=200,
    neg_sampling_ratio=1.0,
)
val_loader = TemporalDataLoader(
    val_data,
    batch_size=200,
    neg_sampling_ratio=1.0,
)
test_loader = TemporalDataLoader(
    test_data,
    batch_size=200,
    neg_sampling_ratio=1.0,
)

neighbors = torch.empty((final_dataset.num_nodes, 10), dtype=torch.long)

neighbor_loader = LastNeighborLoader(final_dataset.num_nodes, size=10, device=device)

train1 = train_loader.__iter__().__next__()
print(train1)
#print(len(np.unique(train1['src'])) + len(np.unique(np.concatenate((train1['dst'], train1['neg_dst'])))))
print(train1['src'])
print(train1['dst'])
print(train1['t'])
print(train1['msg'])
print(train1['neg_dst'])
print(train1['n_id'])

In [ ]:
print(feature_dict)

In [ ]:
import os.path as osp

import torch
from sklearn.metrics import average_precision_score, roc_auc_score
from torch.nn import Linear

from torch_geometric.datasets import JODIEDataset
from torch_geometric.loader import TemporalDataLoader
from torch_geometric.nn import TGNMemory, TransformerConv
from torch_geometric.nn.models.tgn import (
    IdentityMessage,
    LastAggregator,
    LastNeighborLoader,
)

device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')

path = osp.join(osp.abspath(''), 'data', 'JODIE')
dataset = JODIEDataset(path, name='wikipedia')
data = dataset[0]

# For small datasets, we can put the whole dataset on GPU and thus avoid
# expensive memory transfer costs for mini-batches:
data = data.to(device)

train_data, val_data, test_data = data.train_val_test_split(
    val_ratio=0.15, test_ratio=0.15)

print(dataset[0])
print(dataset[0]['y'])
print(dataset[0].num_nodes)

In [ ]:
train_loader = TemporalDataLoader(
    train_data,
    batch_size=200,
    neg_sampling_ratio=1.0,
)
val_loader = TemporalDataLoader(
    val_data,
    batch_size=200,
    neg_sampling_ratio=1.0,
)
test_loader = TemporalDataLoader(
    test_data,
    batch_size=200,
    neg_sampling_ratio=1.0,
)
neighbor_loader = LastNeighborLoader(data.num_nodes, size=10, device=device)

train1 = train_loader.__iter__().__next__()
print(train1)
print(len(np.unique(train1['src'])) + len(np.unique(np.concatenate((train1['dst'], train1['neg_dst'])))))
print(train1['src'])
print(train1['dst'])
print(train1['t'])
print(train1['msg'])
print(train1['neg_dst'])
print(train1['n_id'])

# Train TGN

This just predicts if a link would exist without considering the future time step specifically

In [ ]:
# This code achieves a performance of around 96.60%. However, it is not
# directly comparable to the results reported by the TGN paper since a
# slightly different evaluation setup is used here.
# In particular, predictions in the same batch are made in parallel, i.e.
# predictions for interactions later in the batch have no access to any
# information whatsoever about previous interactions in the same batch.
# On the contrary, when sampling node neighborhoods for interactions later in
# the batch, the TGN paper code has access to previous interactions in the
# batch.
# While both approaches are correct, together with the authors of the paper we
# decided to present this version here as it is more realsitic and a better
# test bed for future methods.

import os.path as osp

import torch
from sklearn.metrics import average_precision_score, roc_auc_score
from torch.nn import Linear

from torch_geometric.data.temporal import TemporalData
from torch_geometric.datasets import JODIEDataset
from torch_geometric.loader import TemporalDataLoader
from torch_geometric.nn import TGNMemory, TransformerConv
from torch_geometric.nn.models.tgn import (
    IdentityMessage,
    LastAggregator,
    LastNeighborLoader,
)

import pandas as pd
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

# Starboard data
events = ['FISH', 'PORT', 'ECTR']
event_dict = dict(zip(events, range(len(events))))
data_events = pd.read_csv('data/Starboard/events.csv')
data_vessels = pd.read_csv('data/Starboard/vessels.csv')
feature_dict = {x['vessel_id']: x['label'] for _, x in data_vessels.iterrows()}

# Initialise inputs 
src = np.zeros((len(data_events)))
dst = np.zeros((len(data_events)))
t = np.zeros((len(data_events)))
# features = (lat_n,lat_s,lon_e,lon_w,ves1,ves2)
features = np.zeros((len(data_events), 4))
y = np.zeros((len(data_events), len(events)))

# convert timesteps
data_events['start_time'] = pd.to_datetime(data_events['start_time'], format='%d/%m/%Y %H:%M')
data_events = data_events.sort_values(by='start_time').reset_index(drop=True)
timesteps = data_events['start_time'].dt.dayofyear

# add fish to events
data_events['event_type'] = data_events['event_type'].fillna('PORT')

for ind, data in data_events.iterrows():
    # Define source and dest array
    src[ind] = data['vessel_id']

    if not np.isnan(data['vessel_id2']):
        dst[ind] = data['vessel_id2']
    elif not np.isnan(data['port_id']):
        dst[ind] = data['port_id']
    else:
        dst[ind] = 0

    # Timestamp
    t[ind] = timesteps[ind]

    # Features
    features[ind] = np.array([data['lat_n'], data['lat_s'], data['lon_e'], data['lon_w']])

    # Even types
    event = data['event_type']
    y[ind][event_dict[event]] = 1

vals, indexes = np.unique(np.concatenate((src, dst)), return_inverse=True)
src, dst = np.split(indexes, 2)
features = np.nan_to_num(features)

src = torch.Tensor(src).type(torch.long)
dst = torch.Tensor(dst).type(torch.long)
t = torch.Tensor(t).type(torch.long)
features = torch.Tensor(features)
y = torch.Tensor(y)

data = TemporalData(src=src, dst=dst, t=t, msg=features, y=y)

# path = osp.join(osp.dirname(osp.realpath(__file__)), 'data', 'JODIE')
# dataset = JODIEDataset(path, name='wikipedia')
# data = dataset[0]

# For small datasets, we can put the whole dataset on GPU and thus avoid
# expensive memory transfer costs for mini-batches:
data = data.to(device)

train_data, val_data, test_data = data.train_val_test_split(
    val_ratio=0.15, test_ratio=0.15)

train_loader = TemporalDataLoader(
    train_data,
    batch_size=100,
    neg_sampling_ratio=1.0,
)
val_loader = TemporalDataLoader(
    val_data,
    batch_size=100,
    neg_sampling_ratio=1.0,
)
test_loader = TemporalDataLoader(
    test_data,
    batch_size=100,
    neg_sampling_ratio=1.0,
)

neighbor_loader = LastNeighborLoader(data.num_nodes, size=10, device=device)


class GraphAttentionEmbedding(torch.nn.Module):
    def __init__(self, in_channels, out_channels, msg_dim, time_enc):
        super().__init__()
        self.time_enc = time_enc
        edge_dim = msg_dim + time_enc.out_channels
        self.conv = TransformerConv(in_channels, out_channels // 2, heads=2,
                                    dropout=0.1, edge_dim=edge_dim)

    def forward(self, x, last_update, edge_index, t, msg):
        rel_t = last_update[edge_index[0]] - t
        rel_t_enc = self.time_enc(rel_t.to(x.dtype))
        edge_attr = torch.cat([rel_t_enc, msg], dim=-1)
        return self.conv(x, edge_index, edge_attr)

class TimeEncoder(torch.nn.Module):
    def __init__(self, out_channels: int):
        super().__init__()
        self.out_channels = out_channels
        self.lin = Linear(1, out_channels)

    def reset_parameters(self):
        self.lin.reset_parameters()

    def forward(self, t: torch.Tensor) -> torch.Tensor:
        return self.lin(t.view(-1, 1)).cos()

class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.lin_src = Linear(in_channels, in_channels)
        self.lin_dst = Linear(in_channels, in_channels)
        self.lin_time = Linear(in_channels, in_channels)
        self.time_enc = TimeEncoder(in_channels)
        self.lin_final = Linear(in_channels, 1)

    def forward(self, z_src, z_dst, t):
        time = self.time_enc(t)
        time = self.lin_time(time).view(z_src.shape)
        h = self.lin_src(z_src) + self.lin_dst(z_dst) + time
        h = h.relu()
        return self.lin_final(h)


memory_dim = time_dim = embedding_dim = 100

memory = TGNMemory(
    data.num_nodes,
    data.msg.size(-1),
    memory_dim,
    time_dim,
    message_module=IdentityMessage(data.msg.size(-1), memory_dim, time_dim),
    aggregator_module=LastAggregator(),
).to(device)

gnn = GraphAttentionEmbedding(
    in_channels=memory_dim,
    out_channels=embedding_dim,
    msg_dim=data.msg.size(-1),
    time_enc=memory.time_enc,
).to(device)

link_pred = LinkPredictor(in_channels=embedding_dim).to(device)

optimizer = torch.optim.Adam(
    set(memory.parameters()) | set(gnn.parameters())
    | set(link_pred.parameters()), lr=0.0001)
criterion_c = torch.nn.BCEWithLogitsLoss()

# Helper vector to map global node indices to local ones.
assoc = torch.empty(data.num_nodes, dtype=torch.long, device=device)


def train():
    memory.train()
    gnn.train()
    link_pred.train()

    memory.reset_state()  # Start with a fresh memory.
    neighbor_loader.reset_state()  # Start with an empty graph.

    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        batch = batch.to(device)

        n_id, edge_index, e_id = neighbor_loader(batch.n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        # Get updated memory of all nodes involved in the computation.
        z, last_update = memory(n_id)
        z = gnn(z, last_update, edge_index, data.t[e_id].to(device),
                data.msg[e_id].to(device))
        time = batch.t.type(torch.cuda.FloatTensor)
        pos_out = link_pred(z[assoc[batch.src]], z[assoc[batch.dst]], time)
        neg_out = link_pred(z[assoc[batch.src]], z[assoc[batch.neg_dst]], time)

        # y_true = torch.stack((torch.ones(len(batch)).to(device), batch.t), dim=1)
        # loss = criterion_c(pos_out, y_true)

        loss = criterion_c(pos_out, torch.ones_like(pos_out))
        loss += criterion_c(neg_out, torch.zeros_like(neg_out))

        # Update memory and neighbor loader with ground-truth state.
        memory.update_state(batch.src, batch.dst, batch.t, batch.msg)
        neighbor_loader.insert(batch.src, batch.dst)

        loss.backward()
        optimizer.step()
        memory.detach()
        total_loss += float(loss) * batch.num_events

    return total_loss / train_data.num_events


@torch.no_grad()
def test(loader):
    memory.eval()
    gnn.eval()
    link_pred.eval()

    torch.manual_seed(12345)  # Ensure deterministic sampling across epochs.

    aps, aucs = [], []
    for batch in loader:
        batch = batch.to(device)

        n_id, edge_index, e_id = neighbor_loader(batch.n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        z, last_update = memory(n_id)
        z = gnn(z, last_update, edge_index, data.t[e_id].to(device),
                data.msg[e_id].to(device))
        time = batch.t.type(torch.cuda.FloatTensor)
        pos_out = link_pred(z[assoc[batch.src]], z[assoc[batch.dst]], time)
        neg_out = link_pred(z[assoc[batch.src]], z[assoc[batch.neg_dst]], time)

        y_pred = torch.cat([pos_out, neg_out], dim=0).sigmoid().cpu()
        y_true = torch.cat(
            [torch.ones(pos_out.size(0)),
             torch.zeros(neg_out.size(0))], dim=0)

        aps.append(average_precision_score(y_true, y_pred))
        aucs.append(roc_auc_score(y_true, y_pred))

        memory.update_state(batch.src, batch.dst, batch.t, batch.msg)
        neighbor_loader.insert(batch.src, batch.dst)
    
    return float(torch.tensor(aps).mean()), float(torch.tensor(aucs).mean())

for epoch in range(1, 51):
    loss = train()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')
    val_ap, val_auc = test(val_loader)
    test_ap, test_auc = test(test_loader)
    print(f'Val AP: {val_ap:.4f}, Val AUC: {val_auc:.4f}')
    print(f'Test AP: {test_ap:.4f}, Test AUC: {test_auc:.4f}')

In [41]:
# Get some predictions to show
ind_map = {vals[i]: i for i in range(len(vals))}
rev_ind_map = {i: vals[i] for i in range(len(vals))}

time_vals = np.arange(max(t)+1, max(t)+25, 5)

permut = np.array(np.meshgrid(vals, vals, time_vals)).T.reshape(-1,3)
pred_src, pred_dst, pred_t = np.hsplit(permut, 3)

pred_src = np.vectorize(ind_map.get)(pred_src)
pred_dst = np.vectorize(ind_map.get)(pred_dst)

pred_src = torch.Tensor(pred_src).type(torch.long)
pred_dst = torch.Tensor(pred_dst).type(torch.long)
pred_t = torch.Tensor(pred_t).type(torch.long)

pred_data = TemporalData(src=pred_src, dst=pred_dst, t=pred_t)

# path = osp.join(osp.dirname(osp.realpath(__file__)), 'data', 'JODIE')
# dataset = JODIEDataset(path, name='wikipedia')
# data = dataset[0]

# For small datasets, we can put the whole dataset on GPU and thus avoid
# expensive memory transfer costs for mini-batches:
pred_data = pred_data.to(device)

batch_size = 200

pred_loader = TemporalDataLoader(
    pred_data,
    batch_size=batch_size
)

final_pred = torch.zeros((pred_src.shape[0], 4)).to(device)

with torch.no_grad():
    memory.eval()
    gnn.eval()
    link_pred.eval()

    torch.manual_seed(12345)  # Ensure deterministic sampling across epochs.

    for ind, batch in enumerate(pred_loader):
        batch = batch.to(device)

        n_id, edge_index, e_id = neighbor_loader(batch.n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        z, last_update = memory(n_id)
        z = gnn(z, last_update, edge_index, data.t[e_id].to(device),
                data.msg[e_id].to(device))
        time = batch.t.type(torch.cuda.FloatTensor)
        pos_out = link_pred(z[assoc[batch.src]], z[assoc[batch.dst]], time)
        cur_pred = torch.stack((pos_out.sigmoid().view(-1, 1), batch.src, batch.dst, time), axis = 1)
        final_pred[batch_size*ind:batch_size*(ind+1)] = cur_pred[:,:,0]

final_dataframe = pd.DataFrame(final_pred.detach().cpu().numpy())
final_dataframe = final_dataframe.replace({1:rev_ind_map, 2:rev_ind_map})

In [ ]:
print(final_dataframe)

## Predicting Time

In [ ]:
import os.path as osp

import torch
from sklearn.metrics import average_precision_score, roc_auc_score, root_mean_squared_error
from torch.nn import Linear

from torch_geometric.data.temporal import TemporalData
from torch_geometric.datasets import JODIEDataset
from torch_geometric.loader import TemporalDataLoader
from torch_geometric.nn import TGNMemory, TransformerConv
from torch_geometric.nn.models.tgn import (
    IdentityMessage,
    LastAggregator,
    LastNeighborLoader,
)

import pandas as pd
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

# Starboard data
events = ['FISH', 'PORT', 'ECTR']
event_dict = dict(zip(events, range(len(events))))
data_events = pd.read_csv('data/Starboard/events.csv')
data_vessels = pd.read_csv('data/Starboard/vessels.csv')
feature_dict = {x['vessel_id']: x['label'] for _, x in data_vessels.iterrows()}

# Initialise inputs 
src = np.zeros((len(data_events)))
dst = np.zeros((len(data_events)))
t = np.zeros((len(data_events)))
# features = (lat_n,lat_s,lon_e,lon_w,ves1,ves2)
features = np.zeros((len(data_events), 4))
y = np.zeros((len(data_events), len(events)))

# convert timesteps
data_events['start_time'] = pd.to_datetime(data_events['start_time'], format='%d/%m/%Y %H:%M')
data_events = data_events.sort_values(by='start_time').reset_index(drop=True)
timesteps = data_events['start_time'].dt.dayofyear

# add fish to events
data_events['event_type'] = data_events['event_type'].fillna('PORT')

for ind, data in data_events.iterrows():
    # Define source and dest array
    src[ind] = data['vessel_id']

    if not np.isnan(data['vessel_id2']):
        dst[ind] = data['vessel_id2']
    elif not np.isnan(data['port_id']):
        dst[ind] = data['port_id']
    else:
        dst[ind] = 0

    # Timestamp
    t[ind] = timesteps[ind]

    # Features
    features[ind] = np.array([data['lat_n'], data['lat_s'], data['lon_e'], data['lon_w']])

    # Even types
    event = data['event_type']
    y[ind][event_dict[event]] = 1

vals, indexes = np.unique(np.concatenate((src, dst)), return_inverse=True)
src, dst = np.split(indexes, 2)
features = np.nan_to_num(features)

src = torch.Tensor(src).type(torch.long)
dst = torch.Tensor(dst).type(torch.long)
t = torch.Tensor(t).type(torch.long)
features = torch.Tensor(features)
y = torch.Tensor(y)

data = TemporalData(src=src, dst=dst, t=t, msg=features, y=y)

# path = osp.join(osp.dirname(osp.realpath(__file__)), 'data', 'JODIE')
# dataset = JODIEDataset(path, name='wikipedia')
# data = dataset[0]

# For small datasets, we can put the whole dataset on GPU and thus avoid
# expensive memory transfer costs for mini-batches:
data = data.to(device)

train_data, val_data, test_data = data.train_val_test_split(
    val_ratio=0.15, test_ratio=0.15)

train_loader = TemporalDataLoader(
    train_data,
    batch_size=100,
    neg_sampling_ratio=1.0,
)
val_loader = TemporalDataLoader(
    val_data,
    batch_size=100,
    neg_sampling_ratio=1.0,
)
test_loader = TemporalDataLoader(
    test_data,
    batch_size=100,
    neg_sampling_ratio=1.0,
)

neighbor_loader = LastNeighborLoader(data.num_nodes, size=10, device=device)


class GraphAttentionEmbedding(torch.nn.Module):
    def __init__(self, in_channels, out_channels, msg_dim, time_enc):
        super().__init__()
        self.time_enc = time_enc
        edge_dim = msg_dim + time_enc.out_channels
        self.conv = TransformerConv(in_channels, out_channels // 2, heads=2,
                                    dropout=0.1, edge_dim=edge_dim)

    def forward(self, x, last_update, edge_index, t, msg):
        rel_t = last_update[edge_index[0]] - t
        rel_t_enc = self.time_enc(rel_t.to(x.dtype))
        edge_attr = torch.cat([rel_t_enc, msg], dim=-1)
        return self.conv(x, edge_index, edge_attr)


class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.lin_src = Linear(in_channels, in_channels)
        self.lin_dst = Linear(in_channels, in_channels)
        self.lin_final1 = Linear(in_channels, 128)
        self.lin_final2 = Linear(128, 2)

    def forward(self, z_src, z_dst):
        h = self.lin_src(z_src) + self.lin_dst(z_dst)
        h = h.relu()
        h = self.lin_final1(h).relu()
        return self.lin_final2(h)


memory_dim = time_dim = embedding_dim = 100

memory = TGNMemory(
    data.num_nodes,
    data.msg.size(-1),
    memory_dim,
    time_dim,
    message_module=IdentityMessage(data.msg.size(-1), memory_dim, time_dim),
    aggregator_module=LastAggregator(),
).to(device)

gnn = GraphAttentionEmbedding(
    in_channels=memory_dim,
    out_channels=embedding_dim,
    msg_dim=data.msg.size(-1),
    time_enc=memory.time_enc,
).to(device)

link_pred = LinkPredictor(in_channels=embedding_dim).to(device)

optimizer = torch.optim.Adam(
    set(memory.parameters()) | set(gnn.parameters())
    | set(link_pred.parameters()), lr=0.0001)

criterion_c = torch.nn.BCEWithLogitsLoss()
criterion_r = torch.nn.MSELoss()
window = 366

# Helper vector to map global node indices to local ones.
assoc = torch.empty(data.num_nodes, dtype=torch.long, device=device)


def train():
    memory.train()
    gnn.train()
    link_pred.train()

    memory.reset_state()  # Start with a fresh memory.
    neighbor_loader.reset_state()  # Start with an empty graph.

    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        batch = batch.to(device)

        n_id, edge_index, e_id = neighbor_loader(batch.n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        # Get updated memory of all nodes involved in the computation.
        z, last_update = memory(n_id)
        z = gnn(z, last_update, edge_index, data.t[e_id].to(device),
                data.msg[e_id].to(device))
        pos_out_full = link_pred(z[assoc[batch.src]], z[assoc[batch.dst]])
        neg_out_full = link_pred(z[assoc[batch.src]], z[assoc[batch.neg_dst]])

        pos_out = pos_out_full[:, 0]
        neg_out = neg_out_full[:, 0]

        reg_out = pos_out_full[:, 1]
        cur_t = batch.t.view(-1) / window

        # y_true = torch.stack((torch.ones(len(batch)).to(device), batch.t), dim=1)
        # loss = criterion_c(pos_out, y_true)

        c_loss = criterion_c(pos_out, torch.ones_like(pos_out))
        c_loss += criterion_c(neg_out, torch.zeros_like(neg_out))
        r_loss = criterion_r(reg_out, cur_t)
        loss = c_loss + r_loss

        # Update memory and neighbor loader with ground-truth state.
        memory.update_state(batch.src, batch.dst, batch.t, batch.msg)
        neighbor_loader.insert(batch.src, batch.dst)

        loss.backward()
        optimizer.step()
        memory.detach()
        total_loss += float(loss) * batch.num_events

    return total_loss / train_data.num_events


@torch.no_grad()
def test(loader):
    memory.eval()
    gnn.eval()
    link_pred.eval()

    torch.manual_seed(12345)  # Ensure deterministic sampling across epochs.

    aps, aucs, rmse = [], [], []
    for batch in loader:
        batch = batch.to(device)

        n_id, edge_index, e_id = neighbor_loader(batch.n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        z, last_update = memory(n_id)
        z = gnn(z, last_update, edge_index, data.t[e_id].to(device),
                data.msg[e_id].to(device))
        pos_out_full = link_pred(z[assoc[batch.src]], z[assoc[batch.dst]])
        neg_out_full = link_pred(z[assoc[batch.src]], z[assoc[batch.neg_dst]])

        pos_out = pos_out_full[:, 0]
        neg_out = neg_out_full[:, 0]

        reg_out = pos_out_full[:, 1].sigmoid().cpu()
        cur_t = batch.t.view(-1) / window
        cur_t = cur_t.sigmoid().cpu()

        y_pred = torch.cat([pos_out, neg_out], dim=0).sigmoid().cpu()
        y_true = torch.cat(
            [torch.ones(pos_out.size(0)),
             torch.zeros(neg_out.size(0))], dim=0)

        aps.append(average_precision_score(y_true, y_pred))
        aucs.append(roc_auc_score(y_true, y_pred))
        rmse.append(root_mean_squared_error(reg_out, cur_t))

        memory.update_state(batch.src, batch.dst, batch.t, batch.msg)
        neighbor_loader.insert(batch.src, batch.dst)
    
    return float(torch.tensor(aps).mean()), float(torch.tensor(aucs).mean()), float(torch.tensor(rmse).mean())

for epoch in range(1, 51):
    loss = train()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')
    val_ap, val_auc, val_rmse = test(val_loader)
    test_ap, test_auc, test_rmse = test(test_loader)
    print(f'Val AP: {val_ap:.4f}, Val AUC: {val_auc:.4f}, Val RMSE: {val_rmse:.4f}')
    print(f'Test AP: {test_ap:.4f}, Test AUC: {test_auc:.4f}, Test RMSE: {test_rmse:.4f}')

In [78]:
ind_map = {vals[i]: i for i in range(len(vals))}
rev_ind_map = {i: vals[i] for i in range(len(vals))}

permut = np.array(np.meshgrid(vals, vals)).T.reshape(-1,2)
pred_src, pred_dst = np.hsplit(permut, 2)

pred_src = np.vectorize(ind_map.get)(pred_src)
pred_dst = np.vectorize(ind_map.get)(pred_dst)

pred_src = torch.Tensor(pred_src).type(torch.long)
pred_dst = torch.Tensor(pred_dst).type(torch.long)
pred_t = torch.ones_like(pred_dst).type(torch.long)

pred_data = TemporalData(src=pred_src, dst=pred_dst, t=pred_t)

# path = osp.join(osp.dirname(osp.realpath(__file__)), 'data', 'JODIE')
# dataset = JODIEDataset(path, name='wikipedia')
# data = dataset[0]

# For small datasets, we can put the whole dataset on GPU and thus avoid
# expensive memory transfer costs for mini-batches:
pred_data = pred_data.to(device)

batch_size = 200

pred_loader = TemporalDataLoader(
    pred_data,
    batch_size=batch_size
)

final_pred = torch.zeros((pred_src.shape[0], 4)).to(device)

with torch.no_grad():
    memory.eval()
    gnn.eval()
    link_pred.eval()

    torch.manual_seed(12345)  # Ensure deterministic sampling across epochs.

    for ind, batch in enumerate(pred_loader):
        batch = batch.to(device)

        n_id, edge_index, e_id = neighbor_loader(batch.n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        z, last_update = memory(n_id)
        z = gnn(z, last_update, edge_index, data.t[e_id].to(device),
                data.msg[e_id].to(device))
        pos_out_full = link_pred(z[assoc[batch.src]], z[assoc[batch.dst]])
        pos_out = pos_out_full[:, :, 0]

        reg_out = pos_out_full[:, :, 1].sigmoid() * window

        cur_pred = torch.stack((pos_out.sigmoid().view(-1, 1), batch.src, batch.dst, reg_out), axis = 1)
        final_pred[batch_size*ind:batch_size*(ind+1)] = cur_pred[:,:,0]

final_dataframe = pd.DataFrame(final_pred.detach().cpu().numpy())
final_dataframe = final_dataframe.replace({1:rev_ind_map, 2:rev_ind_map})
final_dataframe = final_dataframe.sort_values(by=[0], ascending=False)
final_dataframe.to_csv('res/sec_csv.csv')

In [ ]:
print(final_dataframe)

In [4]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.utils
import torch.utils.data
# from torch_geometric import nn as tgnn
from VGRNN.preprocessing import sparse_to_tuple
import scipy.sparse as sp
from torch_geometric.utils import *
import torch_scatter
import inspect
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import tqdm

In [ ]:
adj1 = sp.csr_array([[1, 1, 0, 0], [1, 1, 0, 1], [0, 0, 1, 1], [0, 0, 1, 1]], dtype='int32')
adjs_list = [adj1, adj1]

adj_train_l, train_edges_l, val_edges_l = [], [], []
val_edges_false_l, test_edges_l, test_edges_false_l = [], [], []
edges_list = []

pbar = tqdm.tqdm(total=len(adjs_list))
for i in range(0, len(adjs_list)):
    # Function to build test set with 10% positive links
    # NOTE: Splits are randomized and results might slightly deviate from reported numbers in the paper.
    
    adj = adjs_list[i]
    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()
    # Check that diag is zero:
    assert np.diag(adj.todense()).sum() == 0
    
    # Return upper triangle part of array (assumes bidirectional)
    adj_triu = sp.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]

    # All edges
    edges_all = sparse_to_tuple(adj)[0]
    num_test = int(np.ceil(edges.shape[0]*.10))
    num_val = int(np.ceil(edges.shape[0]*.20))
    
    # Splits all edges to test val train
    all_edge_idx = np.arange(edges.shape[0])
    np.random.shuffle(all_edge_idx)
    val_edge_idx = all_edge_idx[:num_val]
    test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
    test_edges = edges[test_edge_idx]
    val_edges = edges[val_edge_idx]
    train_edges = np.delete(edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0)
    
    edges_list.append(edges)
    
    def ismember(a, b, tol=5):
        rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
        return np.any(rows_close)

    test_edges_false = []
    # Get false test edges 1:1 ratio
    while len(test_edges_false) < len(test_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], edges_all):
            continue
        if test_edges_false:
            if ismember([idx_j, idx_i], np.array(test_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(test_edges_false)):
                continue
        test_edges_false.append([idx_i, idx_j])

    val_edges_false = []
    # Get val edges 1:1 ratio
    while len(val_edges_false) < len(val_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], train_edges):
            continue
        if ismember([idx_j, idx_i], train_edges):
            continue
        if ismember([idx_i, idx_j], val_edges):
            continue
        if ismember([idx_j, idx_i], val_edges):
            continue
        if val_edges_false:
            if ismember([idx_j, idx_i], np.array(val_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(val_edges_false)):
                continue
        val_edges_false.append([idx_i, idx_j])

    assert ~ismember(test_edges_false, edges_all)
    assert ~ismember(val_edges_false, edges_all)
    assert ~ismember(val_edges, train_edges)
    assert ~ismember(test_edges, train_edges)
    assert ~ismember(val_edges, test_edges)

    data = np.ones(train_edges.shape[0])

    # Re-build adj matrix
    adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape)
    adj_train = adj_train + adj_train.T

    adj_train_l.append(adj_train)
    train_edges_l.append(train_edges)
    val_edges_l.append(val_edges)
    val_edges_false_l.append(val_edges_false)
    test_edges_l.append(test_edges)
    test_edges_false_l.append(test_edges_false)
    pbar.update(1)
pbar.close()

In [ ]:
np.where(np.nonzero((adjs_list[1] - adjs_list[0])>0))

In [53]:
edges_pos = np.transpose(np.asarray(np.nonzero((adjs_list[1] - adjs_list[0])>0)))

In [ ]:
edges_pos

In [ ]:
adj_train_l[0].todense(), train_edges_l[0]

In [47]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.utils
import torch.utils.data
from torch.autograd import Variable
# from torch_geometric import nn as tgnn
import scipy.sparse as sp
from torch.nn.parameter import Parameter
import torch.nn.functional as F
import time
from torch_scatter import scatter_mean, scatter_max, scatter_add
from torch_geometric.utils import *
from torch_geometric.nn import MessagePassing
import torch_scatter
import inspect
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import copy
import pickle
import tqdm
import scipy

torch.manual_seed(1123)

logits = torch.rand((4,4))
adj_dense = torch.full((4,4), 1.5)
target_adj_dense = adj_dense.to_sparse_csr()
logits = logits.to_sparse_csr()

In [48]:
logits
temp_size = target_adj_dense.shape[0]
temp_sum = target_adj_dense.sum()
posw = float(temp_size * temp_size - temp_sum) / temp_sum
posw = torch.Tensor([posw])
norm = temp_size * temp_size / float((temp_size * temp_size - temp_sum) * 2)

In [ ]:
torch.sigmoid(logits)

In [ ]:
nll_loss_mat = torch.nn.BCEWithLogitsLoss(reduction = 'none')(logits, target_adj_dense)

In [ ]:
# tensor_mat = from_scipy_sparse_matrix(target_adj_dense)
# fin_tensor_mat = to_torch_sparse_tensor(tensor_mat[0], tensor_mat[1], size=logits.shape[0]).to_dense()
# fin_tensor_mat = fin_tensor_mat.type(torch.float64)
nll_loss_mat = F.binary_cross_entropy_with_logits(input=logits, target=target_adj_dense, pos_weight=posw, reduction='none')
nll_loss = -1 * norm * torch.mean(nll_loss_mat, dim=[0,1])

In [64]:
x_in_list = []
seq_len = 460
num_nodes = 5

x_temp = torch.eye(5)
x_temp = x_temp.expand(seq_len, num_nodes, num_nodes)
x_temp = Variable(x_temp)


In [ ]:
x_in_list = []
seq_len = 460
num_nodes = 5

for i in range(0, seq_len):
    x_temp = torch.tensor(np.eye(num_nodes).astype(np.float32))
    x_in_list.append(torch.tensor(x_temp))

x_in = Variable(torch.stack(x_in_list))

In [ ]:
x_in.shape